Домашнее задание 09 (после Лекции №10, но само задание скорее по теме Лекции №09)

1. Снять видео на камеру.
Выполнить кручение видео со скоростью 10 градусов/с относительно центра
2. Взять повернутое видео после п.1  и вернуть его в исходное состояние.
3. Сконвертируйте видео в mp4 при помощи ffmpeg


Анализ задания: используем библиотеку OpenCV. 
Скачиваем ролик  в папку задания. 
Загружаем видео в переменную Python. 
Обрабатываем ролик, делаем вращение.
Показываем в окне блокнота Юпитер.
Сохраняем на диск.
Затем повернутый ролик поворачиваем в обратную сторону.
Показываем в окне блокнота Юпитер.
Убеждаемся, что после двойного поворота туда-обратно видео имеет вид исходного ролика.
Сохраняем в формате mpeg.


Загрузка видеофайла

In [98]:
quit # reset all variables for repeated run of the program

# loading necessary libraries and the video-file
import cv2
import numpy as np

video_path = 'video_cat_short.mp4'  # file name (and path if necessary)

Функция для просмотра видеофайла

In [99]:
def showVideo(filename):
   """Showing video from filename"""
   myvid = cv2.VideoCapture(filename) # loading into cv2
   # Check if video opened successfully
   if not myvid.isOpened():
        print("Video file not found")
   else:
        print("Video is opened") 
   # showing video:
   while True:
        ret, frame = myvid.read()
        if not ret:
            break
        cv2.imshow('frame', frame)
        p = cv2.waitKey(10) & 0xFF
        if p == ord('q') or p == ord('й'): # pressing "q" (quit) closes the windows. Pity but "й" doesn't work :(            
            break
   myvid.release()    # releasing video channel after viewing is over
   cv2.destroyAllWindows()   # close all windows
   print('Show is over')

Функция, сохраняющая список с кадрами в видеофайл (затем используем ее в функции вращения видео для сохранения результата вращения)

In [100]:
def saveFrameListToFile(framelist, output_path, fps):
    """Saving the frames into the file output_path"""
    global frame_shape_0, frame_shape_1
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # save to mpeg
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_shape_0, frame_shape_1))
    for frame in framelist:
        out.write(frame)
    out.release()
    print('Saving is over')

Функция для вращения видеофайла и сохранения в файл.

In [101]:
def rotateVideo(filename1, filename2, rotation_speed):
   """Loading video from filename1, rotating it by rotation_speed per second, then saving to filename2"""
   # 1. Loading
   myvid = cv2.VideoCapture(filename1) # loading into cv2
   # extracting video frame width and height
   frame_shape_0 = int(myvid.get(cv2.CAP_PROP_FRAME_WIDTH))
   frame_shape_1 = int(myvid.get(cv2.CAP_PROP_FRAME_HEIGHT))
   # extracting FPS value of the video
   fps = myvid.get(cv2.CAP_PROP_FPS) 
   print([frame_shape_0, frame_shape_1])
   # Check if video opened successfully
   if not myvid.isOpened():
        print("Video file not found")
   else:
        print("Video is opened") 
   # 2. Transforming the video to rotate it at the speed of 10 degrees per second
   output_frames = [] # list for frames of new rotated video  
   i = 0 # current frame serial number
   while True:
        ret, frame = myvid.read()
        if not ret:
            break    
        # Calculate the current angle of rotation
        angle = (i / fps) * rotation_speed
        center = (frame.shape[1] // 2, frame.shape[0] // 2)
        # Extracting the rotation matrix
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        # Rotate the frame
        rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))    
        # Append the rotated frame to the output list
        output_frames.append(rotated_frame)    
        i += 1
   # Release the video channel
   myvid.release()
   cv2.destroyAllWindows()   # close all windows
   # 3. Saving the video file
   saveFrameListToFile(output_frames, filename2, fps)  
   

Просмотр исходного видео с котиком. 

In [102]:
showVideo(video_path) 

Video is opened
Show is over


Преобразование видео - вращение со скоростью +10 градусов в секунду.
Затем просмотр видео в отдельном окне.

In [103]:
video_path_rot_1 = 'video_rotated_once.mp4'
rotateVideo(video_path, video_path_rot_1, 10) # calculate rotation and save to file
showVideo(video_path_rot_1) # show rotation video

[1280, 720]
Video is opened
Saving is over
Video is opened
Show is over


Обратное вращение видео (то есть со скоростью минус 10 градусов в секунду).
Демонстрируем результат двойного вращения в отдельном окне.
Ожидаем, что вращения не будет - видео должно быть идентичным исходному ролику.

In [104]:
video_path_rot_2 = 'video_rotated_twice.mp4'
rotateVideo(video_path_rot_1, video_path_rot_2, -10) # calculate new rotation and save to file
showVideo(video_path_rot_2) # show rotation video

[1280, 720]
Video is opened
Saving is over
Video is opened
Show is over


Выводы:
В ходе работы проведено ознакомление с обработкой видео с помощью библиотеки OpenCV.
Был загружен файл с котиком и показан вывод этого исходного ролика.
Затем была рассчитана последовательность кадров для демонстрации ролика с заданной угловой скоростью вращения.
Новый ролик был также показан и сохранен на диск.
После этого файл с вращающимся роликом был загружен и рассчитана последовательность кадров с обратным вращением кадров, то есть двойным вращением - "туда" и "обратно".
Полученный третий ролик также был продемонстрирован и сохранен на диск.
Продемонстрировано, что двойное вращение по описанной методике возвращает кадры ролика к исходному - невращающемуся - виду. Однако черные периферийные области, появившиеся при вписывании в стандартный размер кадра повернутых кадров при повороте, демонстрируют вращение, то есть полной идентичности исходного и дважды повернутого ролика нет.
Кроме того, в ходе долгих экспериментов было выяснено, что на данном этапе, до выяснения всех тонкостей работы с видеоканалами, во избежание трудноустранимых неполадок, типа сохранения преобразованного файла неполной длительности, любую операцию с видео следует заканчивать сохранением преобразованного видео в файл, а для следующей операции (например, нового вращения) извлекать его в видеканал по новой. Попытка использовать один и тот же видеоканал для повторной обработки приводила к плавающим неполадкам при работе, типа сохранения файлов разного размера при повторных прогонах программмы.
